In [5]:
import tensorflow as tf
import os
import numpy as np


BATCH_SIZE = 100
LEARNING_RATE_BASE = 0.01
LEARNING_RATE_DECAY = 0.99
REGULARIZATION_RATE = 0.0001
TRAINING_STEPS = 6000
MOVING_AVERAGE_DECAY = 0.99

INPUT_NODE = 784
OUTPUT_NODE = 10

IMAGE_SIZE = 28
NUM_CHANNELS = 1
NUM_LABELS = 10

CONV1_DEEP = 32
CONV1_SIZE = 5

CONV2_DEEP = 64
CONV2_SIZE = 5

FC_SIZE = 512

MODEL_SAVE_PATH='./convNNModel/'
MODEL_NAME='convNNModel'

class ConvNN:
    def __init__(self,batchSize=BATCH_SIZE,learningRateBase=LEARNING_RATE_BASE, 
                 inputNode=INPUT_NODE, outputNode=OUTPUT_NODE, 
                 regularizationRate=REGULARIZATION_RATE, movingAverageDecay=MOVING_AVERAGE_DECAY,
                 learningRateDecay=LEARNING_RATE_DECAY,trainingSteps=TRAINING_STEPS,
                 imageSize=IMAGE_SIZE,numChannels=NUM_CHANNELS,numLabels=NUM_LABELS,
                 conv1Deep=CONV1_DEEP,conv1Size=CONV1_SIZE,fcSize=FC_SIZE,
                 conv2Deep=CONV2_DEEP,conv2Size=CONV2_SIZE,
                 modelSavePath=MODEL_SAVE_PATH, modelName=MODEL_NAME):
        self.batchSize = batchSize
        self.learningRateBase = learningRateBase
        self.inputNode=inputNode
        self.outputNode = outputNode
        self.regularizationRate=regularizationRate
        self.movingAverageDecay=movingAverageDecay
        self.learningRateDecay=learningRateDecay
        self.trainingSteps=trainingSteps
        self.imageSize=imageSize
        self.numChannels=numChannels
        self.numLabels=numLabels
        self.conv1Deep=conv1Deep
        self.conv1Size=conv1Size
        self.conv2Deep=conv2Deep
        self.conv2Size=conv2Size
        self.fcSize=fcSize
        self.modelSavePath=modelSavePath
        self.modelName=modelName
        
    #also known as train
    def fit(self, trainData):
        tf.reset_default_graph() 
        x = tf.placeholder(tf.float32, [self.batchSize, self.imageSize,
                                        self.imageSize, self.numChannels], name='x-input')
        y_ = tf.placeholder(tf.float32, [None, self.outputNode], name='y-input')

        regularizer = tf.contrib.layers.l2_regularizer(self.regularizationRate)
        y = self.inference(x,False,regularizer)
        global_step = tf.Variable(0, trainable=False)

        # 定义损失函数、学习率、滑动平均操作以及训练过程。
        variable_averages = tf.train.ExponentialMovingAverage(self.movingAverageDecay, global_step)
        variables_averages_op = variable_averages.apply(tf.trainable_variables())
        cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=y, labels=tf.argmax(y_, 1))
        cross_entropy_mean = tf.reduce_mean(cross_entropy)
        loss = cross_entropy_mean + tf.add_n(tf.get_collection('losses'))
        learning_rate = tf.train.exponential_decay(self.learningRateBase,
                                                    global_step,
                                                    trainData.num_examples / self.batchSize, self.learningRateDecay,
                                                    staircase=True)

        train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
        with tf.control_dependencies([train_step, variables_averages_op]):
            train_op = tf.no_op(name='train')

        # 初始化TensorFlow持久化类。
        saver = tf.train.Saver()
        with tf.Session() as sess:
            tf.global_variables_initializer().run()
            for i in range(self.trainingSteps):
                xs, ys = mnist.train.next_batch(self.batchSize)

                reshaped_xs = np.reshape(xs, (
                    self.batchSize,
                    self.imageSize,
                    self.imageSize,
                    self.numChannels))
                _, loss_value, step = sess.run([train_op, loss, global_step], feed_dict={x: reshaped_xs, y_: ys})

                if i % 1000 == 0:
                    print("After %d training step(s), loss on training batch is %g." % (step, loss_value))
        return None
        
        #also known as train
    def predict(self, testData):
        
        return None
        
    def inference(self, input_tensor, train, regularizer):
        with tf.variable_scope('layer1-conv1'):
            conv1_weights = tf.get_variable(
                "weight", [self.conv1Size, self.conv1Size, self.numChannels, self.conv1Deep],
                initializer=tf.truncated_normal_initializer(stddev=0.1))
            conv1_biases = tf.get_variable("bias", [self.conv1Deep], initializer=tf.constant_initializer(0.0))
            conv1 = tf.nn.conv2d(input_tensor, conv1_weights, strides=[1, 1, 1, 1], padding='SAME')
            relu1 = tf.nn.relu(tf.nn.bias_add(conv1, conv1_biases))

        with tf.name_scope("layer2-pool1"):
            pool1 = tf.nn.max_pool(relu1, ksize = [1,2,2,1],strides=[1,2,2,1],padding="SAME")

        with tf.variable_scope("layer3-conv2"):
            conv2_weights = tf.get_variable(
                "weight", [self.conv2Size, self.conv2Size, self.conv1Deep, self.conv2Deep],
                initializer=tf.truncated_normal_initializer(stddev=0.1))
            conv2_biases = tf.get_variable("bias", [self.conv2Deep], initializer=tf.constant_initializer(0.0))
            conv2 = tf.nn.conv2d(pool1, conv2_weights, strides=[1, 1, 1, 1], padding='SAME')
            relu2 = tf.nn.relu(tf.nn.bias_add(conv2, conv2_biases))

        with tf.name_scope("layer4-pool2"):
            pool2 = tf.nn.max_pool(relu2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
            pool_shape = pool2.get_shape().as_list()
            nodes = pool_shape[1] * pool_shape[2] * pool_shape[3]
            reshaped = tf.reshape(pool2, [pool_shape[0], nodes])

        with tf.variable_scope('layer5-fc1'):
            fc1_weights = tf.get_variable("weight", [nodes, self.fcSize],
                                          initializer=tf.truncated_normal_initializer(stddev=0.1))
            if regularizer != None: tf.add_to_collection('losses', regularizer(fc1_weights))
            fc1_biases = tf.get_variable("bias", [self.fcSize], initializer=tf.constant_initializer(0.1))

            fc1 = tf.nn.relu(tf.matmul(reshaped, fc1_weights) + fc1_biases)
            if train: fc1 = tf.nn.dropout(fc1, 0.5)

        with tf.variable_scope('layer6-fc2'):
            fc2_weights = tf.get_variable("weight", [self.fcSize, self.numLabels],
                                          initializer=tf.truncated_normal_initializer(stddev=0.1))
            if regularizer != None: tf.add_to_collection('losses', regularizer(fc2_weights))
            fc2_biases = tf.get_variable("bias", [self.numLabels], initializer=tf.constant_initializer(0.1))
            logit = tf.matmul(fc1, fc2_weights) + fc2_biases
        return logit
        
        

In [ ]:
convDNN = ConvNN()

from tensorflow.examples.tutorials.mnist import input_data

mnist = input_data.read_data_sets("../data/MNIST_data", one_hot=True)

convDNN.fit(mnist.train)

Extracting ../data/MNIST_data/train-images-idx3-ubyte.gz
Extracting ../data/MNIST_data/train-labels-idx1-ubyte.gz
Extracting ../data/MNIST_data/t10k-images-idx3-ubyte.gz
Extracting ../data/MNIST_data/t10k-labels-idx1-ubyte.gz
After 1 training step(s), loss on training batch is 5.66454.
After 1001 training step(s), loss on training batch is 0.793069.
After 2001 training step(s), loss on training batch is 0.731462.
After 3001 training step(s), loss on training batch is 0.686523.
